In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/mywork/Copy of Combined_df.csv
/kaggle/input/capstone-top10/o_hypertension_quantity_per_capita_top_features.csv
/kaggle/input/capstone-top10/o_total_quantity_per_capita_top_features.csv
/kaggle/input/capstone-top10/o_opioids_quantity_per_capita_top_features.csv
/kaggle/input/capstone-top10/o_depression_quantity_per_capita_top_features.csv
/kaggle/input/capstone-top10/o_anxiety_quantity_per_capita_top_features.csv
/kaggle/input/capstone-top10/o_diabetes_quantity_per_capita_top_features.csv
/kaggle/input/capstone-top10/o_asthma_quantity_per_capita_top_features.csv
/kaggle/input/capstone-geojson/2019_spatial_raw_master.geojson
/kaggle/input/capstone-geojson/2020_spatial_raw_master.geojson


In [52]:
import shap
import numpy as np
import pandas as pd
import glob
import os
import geopandas as gpd
from patsy import bs
import statsmodels.formula.api as smf
from patsy import bs, EvalEnvironment
from patsy.builtins import Q
import re

import warnings
warnings.filterwarnings('ignore')

In [ ]:
chunk_size = 5000
df_chunks = pd.read_csv("/kaggle/input/mywork/Copy of Combined_df.csv", chunksize=chunk_size)
df = pd.concat(chunk for chunk in df_chunks)

In [55]:
data = gpd.read_file('/kaggle/input/capstone-geojson/2019_spatial_raw_master.geojson')
data.set_index('geography code', inplace=True)
data1 = gpd.read_file('/kaggle/input/capstone-geojson/2020_spatial_raw_master.geojson')
data1.set_index('geography code', inplace=True)
# Concatenate the two GeoDataFrames
df_geo = pd.concat([data, data1])

In [58]:
medical_outcomes = ['o_diabetes_quantity_per_capita', 'o_hypertension_quantity_per_capita','o_asthma_quantity_per_capita', 'o_depression_quantity_per_capita','o_anxiety_quantity_per_capita', 'o_opioids_quantity_per_capita','o_total_quantity_per_capita']

In [ ]:
# Gets LAD name by removing the last word from LSOA name.
df['lad_name'] = df['lsoa21nm'].apply(lambda x: x.rsplit(' ', 1)[0])
print(df[['lsoa21nm', 'lad_name']])

                        lsoa21nm              lad_name
0            City of London 001A        City of London
1            City of London 001B        City of London
2            City of London 001C        City of London
3            City of London 001E        City of London
4      Barking and Dagenham 016A  Barking and Dagenham
...                          ...                   ...
67503   Vale of White Horse 014H   Vale of White Horse
67504   Vale of White Horse 015G   Vale of White Horse
67505   Vale of White Horse 015H   Vale of White Horse
67506   Vale of White Horse 015I   Vale of White Horse
67507      West Oxfordshire 004H      West Oxfordshire

[67508 rows x 2 columns]


In [ ]:
csv_dir = "/kaggle/input/capstone-top10"
csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))  # like 'diabetes_top10.csv'
top_predictors = {}

In [ ]:
# Reads top 10 predictors from CSV files and stores them for each outcome if valid.
for csv_file in csv_files:
    outcome = os.path.basename(csv_file).replace('_top_features.csv', '')
    if outcome not in df.columns:
        print(f"Warning: Outcome '{outcome_col}' not found in dataset. Skipping.")
        continue
    df_csv = pd.read_csv(csv_file)
    predictors = df_csv["Unnamed: 0"].head(10).tolist()  # Assumes column 'Predictor'
    predictors = [p for p in predictors if p in df.columns]
    if not predictors:
        print(f"Warning: No valid predictors found for {outcome}. Skipping.")
        continue
    top_predictors[outcome] = predictors
    print(f"Top predictors for {outcome}: {predictors}")

Top predictors for o_hypertension_quantity_per_capita: ['c_percent_white', 'e_surface_solar_radiation_downwards_sum', 'c_percent_commute_bus', 'e_surface_thermal_radiation_downwards_sum', 'e_snow_density', 'e_lake_mix_layer_temperature', 'e_evaporation_from_the_top_of_canopy_sum', 'e_surface_net_solar_radiation_sum', 'e_evaporation_from_open_water_surfaces_excluding_oceans_sum', 'c_percent_married_or_in_a_registered_civil_partnership']
Top predictors for o_total_quantity_per_capita: ['c_percent_10_years_or_more', 'c_percent_wfh', 'e_surface_thermal_radiation_downwards_sum', 'e_no2', 'c_percent_commute_bicycle', 'c_percent_commute_bus', 'e_evaporation_from_the_top_of_canopy_sum', 'e_lake_mix_layer_temperature', 'e_temperature_2m', 'e_surface_runoff_sum']
Top predictors for o_opioids_quantity_per_capita: ['c_net_annual_income', 'c_percent_2._professional_occupations', 'e_surface_thermal_radiation_downwards_sum', 'c_percent_mixed', 'c_percent_commute_metro_rail', 'e_no2', 'e_lake_mix_laye

In [62]:
# Collect all unique predictors
all_predictors = set()
for predictors in top_predictors.values():
    all_predictors.update(predictors)
all_predictors = list(all_predictors)
print(f"Total unique predictors: {len(all_predictors)}")

Total unique predictors: 33


In [ ]:
# Calculates the average of selected features and outcomes for each LAD.
agg_columns = all_predictors + medical_outcomes

df_lad = (
    df.groupby('lad_name')[agg_columns]
      .mean()  
      .reset_index()
)

print("Aggregated LAD-level data shape:", df_lad.shape)
print(df_lad.head())

Aggregated LAD-level data shape: (309, 41)
       lad_name  c_percent_student_moved_to_address  \
0          Adur                            0.001513   
1     Allerdale                            0.001302   
2  Amber Valley                            0.001564   
3          Arun                            0.002244   
4      Ashfield                            0.001117   

   e_surface_net_solar_radiation_sum  c_percent_wfh  c_percent_commute_car  \
0                       2.055249e+07       0.307117               0.481294   
1                       2.288438e+08       0.240060               0.559384   
2                       2.513019e+08       0.250396               0.586348   
3                       1.831415e+07       0.251721               0.543190   
4                       2.580425e+08       0.188955               0.606898   

   e_total_aerosol_optical_depth_at_550nm_surface  e_temperature_2m  \
0                                        0.124606        283.813849   
1              

In [ ]:
# Creating the same lad_name column in our GeoDataFrame
df_geo['lad_name'] = df_geo['LSOA21NM'].apply(lambda x: x.rsplit(' ', 1)[0])

# Dissolve by lad_name to get one polygon per LAD
lad_geometries = df_geo.dissolve(by='lad_name')

# Compute centroids
lad_geometries['centroid'] = lad_geometries.geometry.centroid
lad_geometries['centroid_x'] = lad_geometries['centroid'].x
lad_geometries['centroid_y'] = lad_geometries['centroid'].y

# Merge these centroid coordinates back into the aggregated df_lad
df_lad = df_lad.merge(
    lad_geometries[['centroid_x', 'centroid_y']],
    on='lad_name',  
    how='left'
)

In [65]:
df_lad.head(2)

,lad_name,c_percent_student_moved_to_address,e_surface_net_solar_radiation_sum,c_percent_wfh,c_percent_commute_car,e_total_aerosol_optical_depth_at_550nm_surface,e_temperature_2m,c_net_annual_income,c_percent_commute_metro_rail,e_lake_mix_layer_temperature,...,c_percent_2._professional_occupations,o_diabetes_quantity_per_capita,o_hypertension_quantity_per_capita,o_asthma_quantity_per_capita,o_depression_quantity_per_capita,o_anxiety_quantity_per_capita,o_opioids_quantity_per_capita,o_total_quantity_per_capita,centroid_x,centroid_y
0,Adur,0.001513,2.055249e+07,0.307117,0.481294,0.124606,283.813849,33219.047619,0.000473,283.513128,...,0.177747,35.219350,90.354576,12.892678,38.771795,48.131123,31.618506,1029.308259,519832.891078,106424.943881
1,Allerdale,0.001302,2.288438e+08,0.240060,0.559384,0.115197,282.572677,29062.711864,0.000389,282.516248,...,0.156510,40.629214,111.422969,14.896261,41.320104,52.968730,40.982229,1281.491911,320062.312420,534980.812530


In [66]:
lad_geometries.head(2)

,geometry,LSOA21NM,centroid_x,centroid_y,c_percent asian,c_percent black,c_percent mixed,c_percent white,c_percent sikh,c_percent hindu,...,e_snow_and_ice,o_diabetes_quantity_per_capita,o_opioids_quantity_per_capita,o_OME_per_capita,o_total_quantity_per_capita,o_asthma_quantity_per_capita,o_hypertension_quantity_per_capita,o_depression_quantity_per_capita,o_anxiety_quantity_per_capita,centroid
lad_name,,,,,,,,,,,,,,,,,,,,,
Adur,"MULTIPOLYGON (((518163.049 103556.691, 518151....",Adur 002A,519832.891078,106424.943881,0.020801,0.002311,0.030046,0.934515,0.000773,0.000000,...,0.040419,17.366574,19.094030,199.618089,564.413376,6.807405,48.578173,19.829621,25.559192,POINT (519832.891 106424.944)
Allerdale,"MULTIPOLYGON (((298682.700 525222.350, 298820....",Allerdale 007A,320062.312420,534980.812530,0.008368,0.001395,0.008368,0.981869,0.000696,0.002786,...,0.040593,41.120013,37.820984,269.443658,1644.572451,17.086636,131.005476,42.455040,51.946084,POINT (320062.312 534980.813)


In [ ]:
# Removes predictors that are highly correlated with each other to avoid redundancy.
def filter_predictors(predictor_list, df, corr_threshold=0.8):

    selected = []
    for predictor in predictor_list:
        add_predictor = True
        for sel in selected:
            corr_value = df[[predictor, sel]].corr().iloc[0, 1]
            if abs(corr_value) > corr_threshold:
                add_predictor = False
                # Predictor appearing later in ranked order is dropped
                break
        if add_predictor:
            selected.append(predictor)
    return selected

In [68]:
def quote_if_needed(var):
    return f'Q("{var}")' if not re.match(r'^[A-Za-z_][A-Za-z0-9_]*$', var) else var

In [69]:
gam_models = {}  # to store fitted models
selected_predictors_dict = {}  # to store selected predictors per outcome
for outcome in medical_outcomes:
    all_predictors = top_predictors.get(outcome, [])
    predictors_in_data = [var for var in all_predictors if var in df_lad.columns]
    selected_predictors = filter_predictors(predictors_in_data, df_lad, corr_threshold=0.8)
    selected_predictors_dict[outcome] = selected_predictors

    # Quote predictors and outcome where needed
    quoted_outcome = quote_if_needed(outcome)
    quoted_predictors = [quote_if_needed(pred) for pred in selected_predictors]

    # Construct the formula
    formula = (
        f"{quoted_outcome} ~ "
        + " + ".join(quoted_predictors)
        + " + bs(centroid_x, df=5) * bs(centroid_y, df=5)"
    )

    print("\n======================================")
    print(f"Fitting GAM for outcome: {outcome}")
    print("Using predictors:", selected_predictors)
    print("Formula:", formula)
    print("======================================\n")

    # Fit the model
    gam_model = smf.ols(formula=formula, data=df_lad, eval_env=EvalEnvironment.capture(1)).fit()
    gam_models[outcome] = gam_model

    print(gam_model.summary())
    df_lad[f"{outcome}_pred"] = gam_model.predict(df_lad)


Fitting GAM for outcome: o_diabetes_quantity_per_capita
Using predictors: ['e_leaf_area_index_low_vegetation', 'c_net_annual_income', 'c_percent_8._process_plant_and_machine_operatives', 'c_percent_commute_bus', 'c_percent_commute_bicycle', 'c_percent_student_moved_to_address', 'e_lake_mix_layer_depth', 'c_percent_tfw_2km_to_5km', 'c_percent_5._skilled_trades_occupations', 'c_percent_asian']
Formula: o_diabetes_quantity_per_capita ~ e_leaf_area_index_low_vegetation + c_net_annual_income + Q("c_percent_8._process_plant_and_machine_operatives") + c_percent_commute_bus + c_percent_commute_bicycle + c_percent_student_moved_to_address + e_lake_mix_layer_depth + c_percent_tfw_2km_to_5km + Q("c_percent_5._skilled_trades_occupations") + c_percent_asian + bs(centroid_x, df=5) * bs(centroid_y, df=5)

                                  OLS Regression Results                                  
Dep. Variable:     o_diabetes_quantity_per_capita   R-squared:                       0.603
Model:         

In [70]:
df_lad[f"{outcome}_pred"] = gam_model.predict(df_lad)
output_path = f'/kaggle/working/df_lad_with_gam_predictions.csv'
df_lad.to_csv(output_path)

In [71]:
#step_5

!pip install mgwr


In [ ]:
#MGWR execution
from mgwr.gwr import MGWR
from mgwr.sel_bw import Sel_BW
from sklearn.preprocessing import StandardScaler
import numpy as np


medical_outcomes = [
    'o_diabetes_quantity_per_capita',
    'o_hypertension_quantity_per_capita',
    'o_asthma_quantity_per_capita',
    'o_depression_quantity_per_capita',
    'o_anxiety_quantity_per_capita',
    'o_opioids_quantity_per_capita',
    'o_total_quantity_per_capita'
]

# Dictionary to store MGWR models
mgwr_models = {}


In [73]:
for outcome in medical_outcomes:
    print(f"\nProcessing MGWR for: {outcome}")

    # Get selected predictors (already filtered for multicollinearity)
    predictors = selected_predictors_dict.get(outcome, [])
    
    if not predictors:
        print(f"No predictors found for {outcome}, skipping.")
        continue

    # Extract and scale predictors
    X_mgwr = df_lad[predictors].copy()
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_mgwr)

    # Target variable
    y = df_lad[outcome].values.reshape(-1, 1)

    # Coordinates
    coords = df_lad[['centroid_x', 'centroid_y']].values

    # Bandwidth selection
    mgwr_selector = Sel_BW(coords, y, X_scaled, multi=True)
    bw = mgwr_selector.search()
    print(f"Bandwidths selected for {outcome}: {bw}")

    # Fit MGWR model
    mgwr_model = MGWR(coords, y, X_scaled, selector=mgwr_selector)
    mgwr_results = mgwr_model.fit()
    mgwr_models[outcome] = mgwr_model

    # Store coefficients
    mgwr_betas = mgwr_results.params
    df_lad[f'{outcome}_mgwr_intercept'] = mgwr_betas[:, 0]
    for i, col in enumerate(predictors):
        df_lad[f'{outcome}_mgwr_{col}'] = mgwr_betas[:, i + 1]

    # Print coefficient variability
    coef_stds = np.std(mgwr_betas, axis=0)
    print("Coefficient variability (std dev):")
    print(f"  Intercept: {coef_stds[0]:.4f}")
    for i, col in enumerate(predictors):
        print(f"  {col}: {coef_stds[i + 1]:.4f}")



Processing MGWR for: o_diabetes_quantity_per_capita


Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Bandwidths selected for o_diabetes_quantity_per_capita: [ 82. 246.  73. 138. 123.  82. 231. 308. 275. 228. 184.]


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Coefficient variability (std dev):
  Intercept: 3.5658
  e_leaf_area_index_low_vegetation: 0.5422
  c_net_annual_income: 4.1072
  c_percent_8._process_plant_and_machine_operatives: 1.3456
  c_percent_commute_bus: 2.3627
  c_percent_commute_bicycle: 1.6578
  c_percent_student_moved_to_address: 0.7771
  e_lake_mix_layer_depth: 0.1525
  c_percent_tfw_2km_to_5km: 0.3421
  c_percent_5._skilled_trades_occupations: 1.7264
  c_percent_asian: 1.5070

Processing MGWR for: o_hypertension_quantity_per_capita


...

Coefficient variability (std dev):
  Intercept: 0.1346
  c_net_annual_income: 3.3737
  c_percent_2._professional_occupations: 1.2569
  e_surface_thermal_radiation_downwards_sum: 3.4529
  c_percent_mixed: 4.6721
  c_percent_commute_metro_rail: 0.0645
  e_no2: 4.6101
  e_lake_mix_layer_depth: 0.0828
  e_total_aerosol_optical_depth_at_550nm_surface: 1.5290
  e_evaporation_from_the_top_of_canopy_sum: 0.0847

Processing MGWR for: o_total_quantity_per_capita


Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

Bandwidths selected for o_total_quantity_per_capita: [160. 308.  54.  65. 137. 121. 308. 144. 308.]


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Coefficient variability (std dev):
  Intercept: 33.3738
  c_percent_10_years_or_more: 2.0760
  c_percent_wfh: 82.5251
  e_surface_thermal_radiation_downwards_sum: 149.1831
  c_percent_commute_bicycle: 46.4726
  c_percent_commute_bus: 38.2861
  e_evaporation_from_the_top_of_canopy_sum: 7.1022
  e_lake_mix_layer_temperature: 34.5271
  e_surface_runoff_sum: 0.7579


In [ ]:
# Clusters LADs based on MGWR values and saves the results to a CSV file.
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

mgwr_cols = [col for col in df_lad.columns if '_mgwr_' in col]
mgwr_data = df_lad[mgwr_cols].dropna().copy()
lad_names = df_lad.loc[mgwr_data.index, 'lad_name'].values

scaler = StandardScaler()
mgwr_scaled = scaler.fit_transform(mgwr_data)

k = 4
kmeans = KMeans(n_clusters=k, random_state=42)
cluster_labels = kmeans.fit_predict(mgwr_scaled)

df_lad.loc[mgwr_data.index, 'mgwr_region'] = cluster_labels

In [76]:
df_lad['mgwr_region'].value_counts()
df_lad[['lad_name', 'mgwr_region'] + mgwr_cols].head()

export_df = df_lad[['lad_name', 'mgwr_region'] + mgwr_cols]
print(export_df.head())

export_df.to_csv("mgwr_local_coefficients.csv", index=False)
print("MGWR local coefficients saved to 'mgwr_local_coefficients.csv'")



       lad_name  mgwr_region  o_diabetes_quantity_per_capita_mgwr_intercept  \
0          Adur          0.0                                      41.669023   
1     Allerdale          3.0                                      32.564670   
2  Amber Valley          2.0                                      35.336556   
3          Arun          0.0                                      42.275840   
4      Ashfield          2.0                                      35.555053   

   o_diabetes_quantity_per_capita_mgwr_e_leaf_area_index_low_vegetation  \
0                                           0.634895                      
1                                          -0.732608                      
2                                          -0.627625                      
3                                           0.616611                      
4                                          -0.642266                      

   o_diabetes_quantity_per_capita_mgwr_c_net_annual_income  \
0           